# JPEG

Hugo Wendjaneh - Vincent Miqueu-Denjean

BUT3 - Groupe 1

## Introduction
**J**oint **P**hotographic **E**xperts **G**roup ou **JPEG** est une norme qui définit le format d'enregistrement et l'algorithme de décodage pour une représentation numérique compressée d'une image fixe.
La compression **JPEG** repose sur la **suppression** de **données redondantes** et **non perceptibles** dans une image.

## Pourquoi est-ce intéressant de connaître comment ça fonctionne  ?
- La plupart des images de votre téléphone ou de votre caméra sont enregistrées dans le format JPEG.
— Environ 86% des images sur le web sont au format JPEG.
- La plupart des vidéos utilisent le principe de ce format pour être compressées.

Cet algorithme est partout dans notre vie et occupe une place importante dans celui-ci.

## Globalement, qu'est-ce que ça fait ?
L'algorithme de compression va analyser chaque section d'une image, il va trouver et supprimer chaque pixel que nos yeux ne peuvent pas voir facilement.

## Pourquoi ça marche
Les œils humains ne sont pas parfaits, ils ont leurs défauts. JPEG utilise ces défauts pour supprimer les informations que nos yeux ont du mal à percevoir.
Un œil est composé de Tiges qui nous permettent de capter la luminosité, mais aussi, de Cônes pour capter les couleurs (Rouge, Vers, Bleu).
Dans chaque œil, nous avons cent millions de Tiges et seulement 6 millions de Cônes.
Nos yeux sont beaucoup plus réceptifs à la luminosité et à l'obscurité plutot qu'aux couleurs.

## Comment ça marche ?
### Conversion de l'espace couleur
Une image est composée de pixels, ces pixels sont un mélange de rouge, vert et bleu allant de 0 à 255. Le mélange de ces couleurs nous permet d'obtenir une couleur spécifique pour un pixel.

Le processus de **conversion de l'espace couleur** prend ces trois valeurs (Rouge, Vert, Bleu) pour chaque pixel et calcule trois nouvelles valeurs (Luminosité, Chrominance Bleu, Chrominance Rouge). Ce processus est réversible, il est appelé **YCbCr**. Durant ce processus, aucune donnée n'est perdue, c'est juste une nouvelle façon de représenter les couleurs.

In [7]:
# Ouvre le fichier image
from PIL import Image

# Ouvre le fichier image
img = Image.open("image.bmp")

# Transforme l'image en tableau de pixels RGB
pixels = img.load()

# Fonction qui transforme un tuple RGB en tuple YCbCr
def RGBtoYCbCr(RGB):
    R, G, B = RGB
    Y = 0.299 * R + 0.587 * G + 0.114 * B
    Cb = -0.168736 * R - 0.331264 * G + 0.5 * B + 128
    Cr = 0.5 * R - 0.418688 * G - 0.081312 * B + 128
    return (Y, Cb, Cr)

# Fonction qui transforme un tuple YCbCr en tuple RGB
def YCbCrtoRGB(YCbCr):
    Y, Cb, Cr = YCbCr
    R = Y + 1.402 * (Cr - 128)
    G = Y - 0.34414 * (Cb - 128) - 0.71414 * (Cr - 128)
    B = Y + 1.772 * (Cb - 128)
    return (R, G, B)

# Affiche l'image
img.show()

# Transforme chaque pixel du tableau de de tuple RGB en tuple YCbCr
for i in range(img.size[0]):
    for j in range(img.size[1]):
        YCbCr = RGBtoYCbCr(pixels[i,j])
        pixels[i,j] = (int(YCbCr[0]), int(YCbCr[1]), int(YCbCr[2]))

# Affiche l'image après transformation
img.show()

[3084:3103:1223/191443.913540:ERROR:object_proxy.cc(577)] Failed to call method: org.freedesktop.DBus.Properties.Get: object_path= /org/freedesktop/portal/desktop: org.freedesktop.DBus.Error.InvalidArgs: L’interface « org.freedesktop.portal.FileChooser » n’existe pas
[3084:3103:1223/191443.913594:ERROR:select_file_dialog_linux_portal.cc(285)] Failed to read portal version property
[3084:3084:1223/191443.916176:ERROR:policy_logger.cc(156)] :components/enterprise/browser/controller/chrome_browser_cloud_management_controller.cc(161) Cloud management controller initialization aborted as CBCM is not enabled. Please use the `--enable-chrome-browser-cloud-management` command line flag to enable it if you are not using the official Google Chrome build.


### Sous-échantillonnage de la chrominance
Cette méthode permet par la suite de supprimer des données qui ne sont pas perceptibles par nos yeux, comme nous l'avons dit plus tôt nos yeux sont mauvais pour détecter la couleur. Cette méthode est basée sur le fait que nos yeux sont beaucoup plus réceptifs à la luminosité qu'aux couleurs.

Dans cette méthode, nous prenons les valeurs de chrominance bleue et rouge. Par la suite, nous divisons les deux images obtenues en blocs de pixels de 2 par 2. Puis, nous calculons la moyenne de chaque bloc pour supprimer les données redondantes pour faire en sorte que chaque valeur moyenne d'un bloc de quatre pixels soit la même (en occupe un seul). En conséquence, les informations selon lesquelles nos yeux sont incapables de percevoir sont réduites de 75%, soit 1/4 de la taille d'origine, mais la luminosité reste intacte.

In [8]:
# Fonction pour diviser l'image en chrominance bleue et rouge
def extract_blue_chrominance(pixel):
    y, cb, cr = pixel
    return (0, cb, 0)

def extract_red_chrominance(pixel):
    y, cb, cr = pixel
    return (0, 0, cr)

def extract_luminance(pixel):
    y, cb, cr = pixel
    return (y, 0, 0)

# Fonction pour calculer la moyenne des blocs de 2x2
def average_block(pixels, color):
    width, height = img.size
    for i in range(0, width - 1, 2):
        for j in range(0, height - 1, 2):
            # Prendre les pixels dans un bloc de 2x2
            block = [
                pixels[i, j],
                pixels[i + 1, j] if i + 1 < width else pixels[i, j],
                pixels[i, j + 1] if j + 1 < height else pixels[i, j],
                pixels[i + 1, j + 1] if (i + 1 < width and j + 1 < height) else pixels[i, j]
            ]
            # Calculer la moyenne des valeurs de chrominance dans le bloc
            avg_cb = sum(p[1] for p in block) // len(block)
            avg_cr = sum(p[2] for p in block) // len(block)                    
            # Mettre à jour les images de chrominance bleue et rouge
            if color == "blue":
                for x in range(min(2, width - i)):
                    for y in range(min(2, height - j)):
                        blue_pixels[i + x, j + y] = (0, avg_cb, 0)
            elif color == "red":
                for x in range(min(2, width - i)):
                    for y in range(min(2, height - j)):
                        red_pixels[i + x, j + y] = (0, 0, avg_cr)

# Créer une nouvelle image pour la chrominance bleue
blue_chrominance = Image.new("YCbCr", img.size)
blue_pixels = blue_chrominance.load()

# Créer une nouvelle image pour la chrominance rouge
red_chrominance = Image.new("YCbCr", img.size)
red_pixels = red_chrominance.load()

# Créer une nouvelle image pour la luminance
luminance = Image.new("YCbCr", img.size)
luminance_pixels = luminance.load()

# Extraire la chrominance bleue, rouge et la luminance
for i in range(img.size[0]):
    for j in range(img.size[1]):
        blue_pixels[i, j] = extract_blue_chrominance(pixels[i, j])
        red_pixels[i, j] = extract_red_chrominance(pixels[i, j])
        luminance_pixels[i, j] = extract_luminance(pixels[i, j])

# Affichage des images de chrominance bleue et rouge
blue_chrominance.show()
red_chrominance.show()

# Moyenne des blocks de 2x2
average_block(blue_pixels, "blue")
average_block(red_pixels, "red")

# Affichage des images de chrominance bleue et rouge aprés moyenne des block de 2x2
blue_chrominance.show()
red_chrominance.show()

Ouverture dans une session de navigateur existante.
Ouverture dans une session de navigateur existante.
Ouverture dans une session de navigateur existante.


### Transformées en cosinus discrète et Quantification

Par la suite, ces deux étapes suppriment également des informations, mais elles le font en exploitant le fait que nos yeux ne sont pas doués pour percevoir les éléments à autre fréquence dans les images. Mais qu'est-ce que ça veut dire ?

![edge](./edge.bmp)

Sur cette image de l'oreille droite du tigre, nos yeux sont capables de voir les contours de l'oreille, mais pas les détails à l'intérieur de l'oreille. C'est ce que l'on appelle les éléments à haute fréquence. Les éléments à haute fréquence sont des éléments qui changent rapidement dans une image. Les éléments à basse fréquence sont des éléments qui changent lentement dans une image. Nous allons donc utiliser cette information, tromper nos yeux et supprimer les éléments à haute fréquence.

La plupart des images des photographies de nature ou de paysage comportent des parties de l'image qui sont floues et la suppression des variations de couleurs à haute fréquence pour créer des textures plus douces ne sont pas perceptibles par nos yeux. Alors, comment l'algorithme JPEG exploite les nuances de l'œil humain pour supprimer les éléments à haute fréquence ?

L'algorithme JPEG utilise une technique appelée **Transformée en cosinus discrète** ou **DCT**. Cette technique prend un bloc de 8 par 8 pixels et le transforme en un tableau de 8 par 8 valeurs. Chaque valeur représente la quantité de chaque fréquence dans le bloc. La valeur en haut à gauche représente la fréquence la plus basse et la valeur en bas à droite représente la fréquence la plus haute. Nous enlevons 128 à chaque valeur pour que les valeurs soient comprises entre -128 et 127. -128 représente la fréquence la plus basse et 127 la fréquence la plus haute.

![dct](./dct.png)

Dans cette illustration de la DCT, les zones sombres représentent les basses fréquences et les zones claires les hautes fréquences. L'information relative aux hautes fréquences, souvent négligeable pour la perception humaine, est donc regroupée vers les coins de la matrice, tandis que l'information des basses fréquences, plus cruciale pour la perception, est située près du coin supérieur gauche.
Prenons par exemple le deuxième carré en haut à gauche vers le bas. Plus la valeur de celui-ci sera haute, plus il y aura de dégradé vers le bas de blanc à noir dans l'image. Si la valeur est basse, il y aura un dégradé vers le haut de noir à blanc dans l'image. L'ensemble de ces valeurs nous permet de reconstruire l'image.

Voici la formule de la DCT :

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/e06f6ee04c9c879a283edcbb7b1fc18b86fcec5b" alt="DCT" style="background-color: white; padding: 5px; box-sizing: border-box; border-radius: 5px;"/>

u est la fréquence spatiale horizontale, pour les entiers 0 ≤ u < 8

v est la fréquence spatiale verticale, pour les entiers 0 ≤ v < 8

x et y sont les coordonnées horizontales et verticales du pixel, pour 0 ≤ x < 8 et 0 ≤ y < 8

Le processus s'applique sur la chrominance bleue et rouge, mais aussi sur la luminosité.

Les cosinus sont utilisés, car ils sont orthogonaux, ce qui signifie que les valeurs de la DCT sont indépendantes les unes des autres. Il permet de décomposer des signaux en une série de fonctions orthogonales.

Il faut savoir que lorsque l'on réapplique cette formule, nous obtenons les mêmes valeurs que l'original. Cette formule est donc réversible.

Nous allons utiliser le **DCT** pour transformer l'image de pixels en une image de fréquences. Nous allons ensuite utiliser la **Quantification** pour supprimer les fréquences que nos yeux ne peuvent pas voir facilement.

In [9]:
import math

luminance_minus_128 = []
blue_minus_128 = []
red_minus_128 = []

# Sous traction de 128 pour chaque pixel
for i in range(img.size[0]):
    luminance_minus_128.append([])
    blue_minus_128.append([])
    red_minus_128.append([])
    for j in range(img.size[1]):
        luminance_minus_128[i].append(luminance_pixels[i, j][0] - 128)
        blue_minus_128[i].append(blue_pixels[i, j][1] - 128)
        red_minus_128[i].append(red_pixels[i, j][2] - 128)

# Fonction pour calculer le coefficient alpha
def alpha(u):
    return 1 / math.sqrt(2) if u == 0 else 1

# Appliquer la transformation en cosinus discret (DCT) sur un bloc de 8x8
def apply_dct(block):
    dct_block = [[0 for _ in range(8)] for _ in range(8)]  # Initialisation d'une matrice pour stocker les résultats de la DCT
    for x in range(8):
        for y in range(8):
            sum_val = 0
            # Calcul de la DCT pour chaque élément du bloc 8x8
            for i in range(8):
                for j in range(8):
                    sum_val += block[i][j] * math.cos(((2 * i + 1) * x * math.pi) / 16) * math.cos(((2 * j + 1) * y * math.pi) / 16)
            # Stockage du résultat dans la matrice de la DCT après l'application des coefficients alpha
            dct_block[x][y] = round(sum_val * (1 / 4 * alpha(x) * alpha(y)), 2)
    return dct_block

# Appliquer la transformation en cosinus discret (DCT) sur toute la matrice
def discrete_cosine_transform(matrix):
    width = len(matrix)
    height = len(matrix[0])

    dct = []  # Initialisation d'une liste pour stocker les blocs DCT
    for u in range(0, width, 8):
        dct.append([])
        for v in range(0, height, 8):
            # Création de blocs 8x8 à partir de la matrice d'entrée
            block = [[matrix[x][y] if x < width and y < height else 0 for y in range(v, v + 8)] for x in range(u, u + 8)]
            # Application de la DCT sur chaque bloc 8x8
            dct_block = apply_dct(block)
            # Stockage du bloc DCT dans la liste
            dct[-1].append(dct_block)

    # Réorganisation des coefficients DCT dans une seule liste
    dct = [[dct[u][v][x][y] for v in range(len(dct[u])) for y in range(8)] for u in range(len(dct)) for x in range(8)]
    return dct

            

# DCT de la luminance
luminance_dct = discrete_cosine_transform(luminance_minus_128)
# DCT de la chrominance bleue
blue_dct = discrete_cosine_transform(blue_minus_128)
# DCT de la chrominance rouge
red_dct = discrete_cosine_transform(red_minus_128)

print("Exemple d'une ligne de de pixel sous forme de fréquence de la luminance: ")
print(luminance_dct[0])
print("PS: Nous avons arrondis les valeurs pour une meilleur lisibilité")

Ouverture dans une session de navigateur existante.
Ouverture dans une session de navigateur existante.
Exemple d'une ligne de de pixel sous forme de fréquence de la luminance: 
[576.0, -7.65, -1.71, 2.43, -2.0, -0.63, 0.44, 0.9, 594.62, -4.19, 1.51, -4.14, 0.38, -0.14, 0.05, -0.77, 606.25, 8.65, -11.52, 0.59, -4.0, 0.41, -0.37, -0.07, 406.75, 95.15, -11.96, 5.73, -1.25, 2.09, 1.17, 0.51, 75.37, 70.37, 9.36, 8.97, 3.88, 0.27, 0.05, 0.49, -99.0, 49.26, -5.19, 3.42, 1.5, 3.0, 0.53, 1.03, -361.12, 83.9, 2.65, 6.0, 2.12, 3.8, -1.77, 0.29, -551.75, 28.16, 1.63, 3.88, -4.75, -0.94, 0.68, 0.44, -559.0, -29.02, 7.23, -3.72, 2.0, -2.1, -0.83, 1.04, -291.5, -124.33, 4.18, -7.56, 3.5, 0.21, 7.47, -1.16, 129.37, -85.19, -45.69, 5.59, -11.87, 0.56, -6.11, -1.25, 91.5, -93.1, 73.4, 17.74, 13.5, 2.74, 0.17, 0.43, 162.75, 129.27, -3.55, -7.41, -3.25, -0.87, -5.3, 0.33, -64.87, 3.4, 7.6, 7.21, 2.37, -0.04, 1.04, -0.56, -76.62, 22.93, -5.84, 1.39, -3.37, 0.11, -5.29, -0.1, -251.87, 58.88, 7.66, 11.43, -

<div style="background-color: yellow; color: black; padding: 5px; border-radius: 5px; box-sizing: border-box;">PS : Le programme ci-dessus n'est pas optimisé, en revanche, il est plus facile à comprendre. Nous aurions pu faire moins de boucles et stocker les valeurs des cosinus dans un tableau pour éviter de les recalculer à chaque fois.</div>

À ce stade, nous avons créé une variable qui représente les valeurs du DCT sous forme matricielle pour chaque chrominance et luminosité.

Maintenant, nous devons appliquer la **Quantification** sur les valeurs du DCT pour chaque chrominance et luminosité.
La **Quantification** est une technique qui permet de supprimer les fréquences que nos yeux ne peuvent pas voir facilement.

Il existe plusieurs matrices pour la **Quantification**. Ces matrices définissent le niveau de compression de l'image. Plus cette matrice va permettre d'avoir des 0, plus l'image sera compressée, et moins la qualité de l'image sera bonne.

Voici la matrice qui permet de faire une compression de 50% comme fourni par la norme JPEG :

<table>
    <tr>
        <td>16</td>
        <td>11</td>
        <td>10</td>
        <td>16</td>
        <td>24</td>
        <td>40</td>
        <td>51</td>
        <td>61</td>
    </tr>
    <tr>
        <td>12</td>
        <td>12</td>
        <td>14</td>
        <td>19</td>
        <td>26</td>
        <td>58</td>
        <td>60</td>
        <td>55</td>
    </tr>
    <tr>
        <td>14</td>
        <td>13</td>
        <td>16</td>
        <td>24</td>
        <td>40</td>
        <td>57</td>
        <td>69</td>
        <td>56</td>
    </tr>
    <tr>
        <td>14</td>
        <td>17</td>
        <td>22</td>
        <td>29</td>
        <td>51</td>
        <td>87</td>
        <td>80</td>
        <td>62</td>
    </tr>
    <tr>
        <td>18</td>
        <td>22</td>
        <td>37</td>
        <td>56</td>
        <td>68</td>
        <td>109</td>
        <td>103</td>
        <td>77</td>
    </tr>
    <tr>
        <td>24</td>
        <td>35</td>
        <td>55</td>
        <td>64</td>
        <td>81</td>
        <td>104</td>
        <td>113</td>
        <td>92</td>
    </tr>
    <tr>
        <td>49</td>
        <td>64</td>
        <td>78</td>
        <td>87</td>
        <td>103</td>
        <td>121</td>
        <td>120</td>
        <td>101</td>
    </tr>
    <tr>
        <td>72</td>
        <td>92</td>
        <td>95</td>
        <td>98</td>
        <td>112</td>
        <td>100</td>
        <td>103</td>
        <td>99</td>
    </tr>
</table>

Pour effectuer la **Quantification**, nous prenons toujours de bloc de 8 par 8 pixels. Nous divisons les valeurs du DCT par les valeurs de la matrice de **Quantification**. Nous arrondissons les valeurs obtenues à l'entier le plus proche. Nous obtenons ainsi une nouvelle matrice de 8 par 8 valeurs. Nous allons ensuite utiliser cette matrice pour reconstruire l'image.

In [10]:
quantization_table_fifty_percent = [
    [16, 11, 10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 58, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99]
]

def quantize(dct, quantization_table):
    quantized_dct = []
    for i in range(len(dct)):
        quantized_dct.append([])
        for j in range(len(dct[i])):
            # Division de chaque coefficient DCT par le coefficient de quantification correspondant
            quantized_dct[i].append(round(dct[i][j] / quantization_table[i % 8][j % 8]))
    return quantized_dct

luminance_quantization = quantize(luminance_dct, quantization_table_fifty_percent)
blue_quantization = quantize(blue_dct, quantization_table_fifty_percent)
red_quantization = quantize(red_dct, quantization_table_fifty_percent)

print("Exemple d'une ligne de de pixel sous forme de fréquence de la luminance aprés quantification: ")
print(luminance_quantization[0])

Exemple d'une ligne de de pixel sous forme de fréquence de la luminance aprés quantification: 
[36, -1, 0, 0, 0, 0, 0, 0, 37, 0, 0, 0, 0, 0, 0, 0, 38, 1, -1, 0, 0, 0, 0, 0, 25, 9, -1, 0, 0, 0, 0, 0, 5, 6, 1, 1, 0, 0, 0, 0, -6, 4, -1, 0, 0, 0, 0, 0, -23, 8, 0, 0, 0, 0, 0, 0, -34, 3, 0, 0, 0, 0, 0, 0, -35, -3, 1, 0, 0, 0, 0, 0, -18, -11, 0, 0, 0, 0, 0, 0, 8, -8, -5, 0, 0, 0, 0, 0, 6, -8, 7, 1, 1, 0, 0, 0, 10, 12, 0, 0, 0, 0, 0, 0, -4, 0, 1, 0, 0, 0, 0, 0, -5, 2, -1, 0, 0, 0, 0, 0, -16, 5, 1, 1, 0, 0, 0, 0, -33, 3, 4, 0, 0, 0, 0, 0, -27, 0, -2, 0, 0, 0, 0, 0, -29, 0, 1, -1, 0, 0, 0, 0, -10, -13, -1, 0, 0, 0, 0, 0, -2, -1, 1, 0, 0, 0, 0, 0, 2, -7, 3, 0, 0, 0, 0, 0, 9, 7, -6, 1, 0, 0, 0, 0, -18, 3, 2, 0, 0, 0, 0, 0, -13, -3, -1, 0, 0, 0, 0, 0, -10, 0, 0, 0, 0, 0, 0, 0, -16, 5, 0, 0, 0, 0, 0, 0, -16, -1, 0, -1, 0, 0, 0, 0, -4, -8, 0, 0, 0, 0, 0, 0, 16, -7, -1, 0, 0, 0, 0, 0, 25, 0, -1, 0, 0, 0, 0, 0, 26, 6, -3, 0, 0, 0, 0, 0, 11, 3, 1, 0, 0, 0, 0, 0, 7, 0, 2, 0, 0, 0, 0, 0, 20, -10, 0, 0, 0,

Vous pouvez voir que les valeurs de la matrice de **Quantification** sont plus grandes dans le coin supérieur gauche et plus petites dans les coins inférieurs droits. Cela signifie que les fréquences basses sont plus susceptibles d'être conservées que les fréquences élevées. Cela permet donc de supprimer de l'information que nos yeux ne peuvent pas voir facilement.

Nous allons par la suite pouvoir simplement recoder cette image en marquant non pas chaque case de la matrice, mais plus tôt les cases de la matrice et combien de fois, elles se répètent. (Run length encoding)

Exemple:
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1] => (1, 10) car 1 se répète 10 fois.

Mais nous avons un problème parce que les valeurs qui se répètent le plus sont des zéros qui sont en bas à droite de la matrice. Nous allons donc utiliser une technique appelée **ZigZag** (Entropy coding) pour réorganiser les valeurs de la matrice.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/JPEG_ZigZag.svg/220px-JPEG_ZigZag.svg.png" alt="ZigZag"/>

In [11]:
def zigzag(matrix):
    zigzag_list = []
    rows = len(matrix)
    cols = len(matrix[0])

    for i in range(rows + cols - 1):
        if i % 2 == 0:
            # Déterminer le point de départ de la ligne
            if i < rows:
                start_row, start_col = i, 0
            else:
                start_row, start_col = rows - 1, i - rows + 1
            # Parcourir la ligne
            while start_row >= 0 and start_col < cols:
                zigzag_list.append(matrix[start_row][start_col])
                start_row -= 1
                start_col += 1
        else:
            # Déterminer le point de départ de la ligne
            if i < cols:
                start_row, start_col = 0, i
            else:
                start_row, start_col = i - cols + 1, cols - 1
            # Parcourir la ligne
            while start_row < rows and start_col >= 0:
                zigzag_list.append(matrix[start_row][start_col])
                start_row += 1
                start_col -= 1

    return zigzag_list

luminance_zigzag = zigzag(luminance_quantization)
red_zigzag = zigzag(red_quantization)
blue_zigzag = zigzag(blue_quantization)
print("Exemple d'une partie d'une ligne de pixel sous forme de fréquence de la luminance après quantification et zigzag : ")
print(luminance_zigzag[0:50])


Exemple d'une partie d'une ligne de pixel sous forme de fréquence de la luminance après quantification et zigzag : 
[36, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 36, 0, 0, 0, 0, 0, 0, 0, 37, 0, 0, 0, 0, 0]


### Longueur d'exécution

Nous allons maintenant utiliser la technique **Run length encoding** que nous avons vue plus tôt pour encoder les valeurs de la matrice.

In [12]:
def run_length_encoding(data):
    if not data:
        return []

    encoded_data = []
    count = 1
    current_value = data[0]

    for i in range(1, len(data)):
        if data[i] == current_value:
            count += 1
        else:
            encoded_data.append((count, current_value))
            count = 1
            current_value = data[i]

    # Ajouter la dernière valeur
    encoded_data.append((count, current_value))

    return encoded_data

luminance_rle = run_length_encoding(luminance_zigzag)
red_rle = run_length_encoding(red_zigzag)
blue_rle = run_length_encoding(blue_zigzag)
print("Exemple partie d'une ligne de pixel sous forme de fréquence de la luminance après quantification, zigzag et RLE : ")
print(luminance_rle[0:50])

Exemple partie d'une ligne de pixel sous forme de fréquence de la luminance après quantification, zigzag et RLE : 
[(1, 36), (1, -1), (34, 0), (1, 36), (7, 0), (1, 37), (8, 0), (1, -1), (82, 0), (1, 37), (7, 0), (1, 37), (7, 0), (1, 38), (1, 1), (7, 0), (1, -1), (27, 0), (1, -1), (110, 0), (1, 38), (7, 0), (1, 38), (7, 0), (1, 38), (7, 0), (1, 25), (1, 9), (1, -1), (6, 0), (1, 1), (7, 0), (1, -1), (27, 0), (1, -1), (7, 0), (1, -1), (150, 0), (1, 38), (7, 0), (1, 39), (7, 0), (1, 38), (7, 0), (1, 27), (7, 0), (1, 5), (1, 6), (1, -1), (6, 0)]


# Huffman Encoding

L'encodage de Huffman est une méthode de compression de données sans perte, inventée par David A. Huffman en 1952. Cette technique est largement utilisée dans la compression de fichiers, les protocoles de transmission de données et d'autres applications où la réduction de la taille des données est nécessaire, comme le <span style="color: orange;">JPEG</span>.

L'idée fondamentale de l'encodage de Huffman est d'attribuer des codes binaires de longueurs variables aux symboles d'un ensemble de données, en utilisant des courts codes pour les symboles fréquents et des codes longs pour les symboles moins fréquents. Ainsi, les symboles fréquents seront représentés par moins de bits, ce qui contribue à la compression des données.

Il convient de noter que bien que l'encodage de Huffman soit efficace pour des ensembles de données avec des schémas de fréquences spécifiques, il peut ne pas être optimal dans tous les cas. Pour des ensembles de données différents, d'autres méthodes de compression peuvent être plus adaptées.

Voici un exemple de l'encodage de Huffman sous forme d'arbre binaire :

![Huffman](./huffman.jpg)

# Données dans un fichier JPEG

![Meta](./meta.png)

Voici comment sont stockées les données dans un fichier JPEG.

Le tout premier marqueur qui nous intéresse est FF D8. Il nous indique qu'il s'agit du début de l'image. Si nous ne le voyons pas, nous pouvons supposer qu'il s'agit d'un autre fichier. Un autre marqueur tout aussi important est FF D9. Il nous indique que nous avons atteint la fin d'un fichier image. Chaque marqueur est immédiatement suivi d'un spécificateur de longueur qui donnera la longueur du segment de marqueur. Les marqueurs de début et de fin de fichier image ont toujours une longueur de deux octets chacun.

Voici la liste des marqueurs qui nous sont utiles pour encoder une image JPEG :
- 0xffd8: Le début du JPEG
- 0xffe0: En-tête par défaut de l'application
- 0xffdb: Définition de la table de quantification
- 0xffc0: Début du cadre
- 0xffc4: Définition de la table de Huffman
- 0xffda: Début de l'analyse des numéros de balayage
- 0xffd9: Fin du JPEG

# Conclusion

En conclusion, le format de fichier JPEG (Joint Photographic Experts Group) a révolutionné la manière dont les images sont stockées, partagées et utilisées à travers le monde. Grâce à son algorithme de compression efficace, le JPEG offre une combinaison unique de qualité visuelle acceptable et de taille de fichier réduite, ce qui en fait le choix préféré pour la plupart des applications sur le web, les réseaux sociaux, le stockage d'images et bien plus encore.

L'approche de compression avec perte du JPEG a permis de réduire significativement la taille des fichiers image tout en conservant une qualité visuelle souvent satisfaisante pour les utilisations courantes. Cependant, il est important de noter que cette compression peut engendrer une perte de qualité. En effet, les données supprimées ne peuvent pas être récupérées, ce qui peut entraîner une dégradation de la qualité visuelle, en particulier lorsque le taux de compression est élevé.

Dans l'ensemble, le JPEG demeure un pilier essentiel dans le domaine de l'imagerie numérique, offrant une solution polyvalente pour la compression d'images, facilitant ainsi leur utilisation, leur partage et leur stockage à l'ère numérique.